In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
from openpyxl import Workbook
import xlsxwriter
import string
import re as re
from datetime import datetime as dt

## Programación

In [3]:
#Programación VFE - primer semestre
primersemestre = pd.read_excel('C:/Users/1032452285/OneDrive - Camara de Comercio de Bogota/Programación/PROGRAMACIÓN VFE I SEMESTRE 2023.xlsx', sheet_name='PROGRAMACIÓN',header=1, skiprows=[0,1])
primersemestre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1706 entries, 0 to 1705
Data columns (total 53 columns):
 #   Column                                          Non-Null Count  Dtype         
---  ------                                          --------------  -----         
 0   MES                                             1406 non-null   object        
 1   FECHA (lista desplegable)                       1406 non-null   datetime64[ns]
 2   DÍA                                             1406 non-null   object        
 3   HORARIO (lista desplegable)                     1406 non-null   object        
 4   JORNADA                                         1706 non-null   object        
 5   TIPO DE HORARIO                                 1706 non-null   object        
 6   DURACIÓN DE LA SESIÓN                           1406 non-null   object        
 7   LUGAR DE PRESTACIÓN                             1406 non-null   object        
 8   BOGOTÁ/REGIÓN                                   

c:\Users\1032452285\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [4]:
#Si solo hay primer semestre
consolidado=primersemestre

In [5]:
consolidado=consolidado.rename(columns = {"CÓDIGO SERVICIO": "cod_servicio",
                                  "Código de agenda": "cod_agenda",
                                  "NOMBRE DE LA AGENDA (Evento masivo)": "nombre_agenda",
                                  "NOMBRE DEL SERVICIO (lista desplegable)":"nombre_servicio",
                                  "CONFERENCISTA ACTUAL (lista desplegable)":"nombre_facilitador",
                                  "PROGRAMA (lista desplegable)":"programa",
                                  "WEBINAR/REUNION ZOOM (lista desplegable)":"es_grabacion",
                                  "ÁREA TEMÁTICA":"area_tematica"})

In [6]:
#Quitar espacios sin número de agenda
consolidado=consolidado[consolidado['cod_agenda'].notna()]
consolidado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1243 entries, 0 to 1406
Data columns (total 53 columns):
 #   Column                                          Non-Null Count  Dtype         
---  ------                                          --------------  -----         
 0   MES                                             1243 non-null   object        
 1   FECHA (lista desplegable)                       1243 non-null   datetime64[ns]
 2   DÍA                                             1243 non-null   object        
 3   HORARIO (lista desplegable)                     1243 non-null   object        
 4   JORNADA                                         1243 non-null   object        
 5   TIPO DE HORARIO                                 1243 non-null   object        
 6   DURACIÓN DE LA SESIÓN                           1243 non-null   object        
 7   LUGAR DE PRESTACIÓN                             1243 non-null   object        
 8   BOGOTÁ/REGIÓN                                   

In [7]:
#Quitar duplicados
consolidado=consolidado.drop_duplicates(subset=['cod_agenda'], keep='last')
consolidado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1190 entries, 0 to 1406
Data columns (total 53 columns):
 #   Column                                          Non-Null Count  Dtype         
---  ------                                          --------------  -----         
 0   MES                                             1190 non-null   object        
 1   FECHA (lista desplegable)                       1190 non-null   datetime64[ns]
 2   DÍA                                             1190 non-null   object        
 3   HORARIO (lista desplegable)                     1190 non-null   object        
 4   JORNADA                                         1190 non-null   object        
 5   TIPO DE HORARIO                                 1190 non-null   object        
 6   DURACIÓN DE LA SESIÓN                           1190 non-null   object        
 7   LUGAR DE PRESTACIÓN                             1190 non-null   object        
 8   BOGOTÁ/REGIÓN                                   

# Asistencias (Actividad-sesion)

In [8]:
#Base de asistencias
asistencias = pd.read_excel('C:/Users/1032452285/OneDrive - Camara de Comercio de Bogota/Documentos - Gestión Analítica VFE/Mensuales/5. Cierre de cifras/1. Archivos de soporte/Reporte Asistencias.xlsx')
asistencias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46850 entries, 0 to 46849
Data columns (total 40 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   nombre_usuario                       31136 non-null  object        
 1   numero_documento                     46850 non-null  object        
 2   telefono                             25089 non-null  object        
 3   celular                              26790 non-null  object        
 4   correo_electronico                   30631 non-null  object        
 5   nombre_municipio                     31055 non-null  object        
 6   razon_social                         31136 non-null  object        
 7   nit_empresa                          13340 non-null  float64       
 8   codigo_actividad                     46850 non-null  object        
 9   nombre_actividad                     46850 non-null  object        
 10  tipo_activ

In [9]:
asistencias[['fechaFin','fechaInicio']]=asistencias[['fecha_fin_actividad','fecha_inicio_actividad']].apply(pd.to_datetime,format='%Y-%m-%d')
asistencias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46850 entries, 0 to 46849
Data columns (total 42 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   nombre_usuario                       31136 non-null  object        
 1   numero_documento                     46850 non-null  object        
 2   telefono                             25089 non-null  object        
 3   celular                              26790 non-null  object        
 4   correo_electronico                   30631 non-null  object        
 5   nombre_municipio                     31055 non-null  object        
 6   razon_social                         31136 non-null  object        
 7   nit_empresa                          13340 non-null  float64       
 8   codigo_actividad                     46850 non-null  object        
 9   nombre_actividad                     46850 non-null  object        
 10  tipo_activ

In [10]:
#Solo nos quedamos con 2023 ###CAMBIA MENSUAL
asistencias=asistencias[(asistencias['fechaInicio'].dt.year ==2023)]
asistencias=asistencias[(asistencias['fechaInicio'].dt.month<=3)]

In [11]:
#Eliminar si no es VFE 
asistenciasVFE=asistencias.loc[asistencias['linea_accion_nombre']=="VICEPRESIDENCIA DE FORTALECIMIENTO EMPRESARIAL"]

In [12]:
#Eliminar si Gerencia
programasexcluidos = ['GERENCIA DE FORMACION EMPRESARIAL']
asistenciasVFE=asistenciasVFE.loc[~asistenciasVFE['linea_trabajo_nombre'].isin(programasexcluidos)]

In [13]:
#Eliminar formalización
formalizacion=['CRM-00002','CRM-00004']
asistenciasVFE=asistenciasVFE.loc[~asistenciasVFE['codigo_actividad'].isin(formalizacion)]

In [14]:
#Quitar asesorías individuales
asesorias=[['Asesoría Individual']]
asistenciasVFE=asistenciasVFE.loc[~asistenciasVFE['tipo_actividad'].isin(asesorias)]

In [15]:
#Quitar duplicados
asistenciasVFE=asistenciasVFE.drop_duplicates(subset=['codigo_agenda','numero_documento','sesion'], keep='last')

In [16]:
asistenciasVFE['linea_trabajo_nombre'].value_counts()

SOLUCIONES EMPRESARIALES                    25916
EMPRENDIMIENTO                               5626
CONSOLIDACIÓN Y ESCALAMIENTO EMPRESARIAL     3910
FORMALIZACIÓN Y CRECIMIENTO EMPRESARIAL      3687
FORO DE PRESIDENTES Y MENTORÍAS              2579
INNOVACION                                   1934
GERENCIA DE ARTICULACION MACROSECTORIAL         6
Name: linea_trabajo_nombre, dtype: int64

In [17]:
asistenciasVFE['control_asistencia'].value_counts()

Asistio    43658
Name: control_asistencia, dtype: int64

In [18]:
asistenciasVFE['control_asistencia'] = ["1" if s =="Asistio" else "0" for s in asistenciasVFE['control_asistencia']]

# Cruce con programación

In [19]:
asistenciasVFEpr = asistenciasVFE.merge(consolidado, left_on='codigo_agenda', right_on='cod_agenda', how='outer', indicator = True)

In [20]:
asistenciasVFEpr=asistenciasVFEpr[asistenciasVFEpr['codigo_agenda'].notna()]
asistenciasVFEpr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43658 entries, 0 to 43657
Data columns (total 96 columns):
 #   Column                                          Non-Null Count  Dtype         
---  ------                                          --------------  -----         
 0   nombre_usuario                                  27953 non-null  object        
 1   numero_documento                                43658 non-null  object        
 2   telefono                                        22864 non-null  object        
 3   celular                                         23728 non-null  object        
 4   correo_electronico                              27449 non-null  object        
 5   nombre_municipio                                27882 non-null  object        
 6   razon_social                                    27953 non-null  object        
 7   nit_empresa                                     11340 non-null  float64       
 8   codigo_actividad                              

**Agendas faltantes**

In [21]:
#Para saber cuantos faltan (left only)
asistenciasVFEpr['_merge'].value_counts()

both          43610
left_only        48
right_only        0
Name: _merge, dtype: int64

In [145]:
agendasfaltantes=asistenciasVFEpr.loc[asistenciasVFEpr['_merge']=='left_only']

In [147]:
agendasfaltantes['Inscritos'] = agendasfaltantes.groupby(['codigo_agenda'])['numero_documento'].transform('count')

C:\Users\1032452285\AppData\Local\Temp\ipykernel_6296\4037912825.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agendasfaltantes['Inscritos'] = agendasfaltantes.groupby(['codigo_agenda'])['numero_documento'].transform('count')


In [149]:
agendasfaltantes=agendasfaltantes.drop_duplicates(subset=['codigo_agenda'], keep='last')
agendasfaltantes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1 entries, 22387 to 22387
Data columns (total 97 columns):
 #   Column                                          Non-Null Count  Dtype         
---  ------                                          --------------  -----         
 0   nombre_usuario                                  1 non-null      object        
 1   numero_documento                                1 non-null      object        
 2   telefono                                        1 non-null      object        
 3   celular                                         1 non-null      object        
 4   correo_electronico                              1 non-null      object        
 5   nombre_municipio                                1 non-null      object        
 6   razon_social                                    1 non-null      object        
 7   nit_empresa                                     1 non-null      float64       
 8   codigo_actividad                              

In [ ]:
# Si las agendas NO son asesorías individuales, pasarlas a Excel y pedir apoyo de Adriana
agendasfaltantes.to_excel('C:/Users/1032452285/OneDrive - Camara de Comercio de Bogota/Documentos - Gestión Analítica VFE/Mensuales/agendasfaltantes.xlsx', index = False)

In [23]:
# Si las agendas son asesorías individuales, eliminarlas
_merge=['both']
asistenciasVFEpr=asistenciasVFEpr.loc[asistenciasVFEpr['_merge'].isin(_merge)]
asistenciasVFEpr['_merge'].value_counts()

both          43610
left_only         0
right_only        0
Name: _merge, dtype: int64

# Base asistencias corregida

In [24]:
actividades = asistenciasVFEpr.rename(columns={'nombre_usuario':'NOMBRE_USUARIO',
                                             'numero_documento':'NUM_DOCUMENTO',
                                             'telefono':'TELEFONO_UNO',
                                             'celular':'CELULAR',
                                             'correo_electronico':'CORREO_ELECTRONICO',
                                             'nombre_municipio':'NOMBRE_MUNICIPIO',
                                             'razon_social':'RAZON_SOCIAL',
                                             'nit_empresa':'NIT_EMPRESA',
                                             'codigo_actividad':'CODIGO_ACTIVIDAD',
                                             'nombre_actividad':'NOMBRE_ACTIVIDAD',
                                             'tipo_actividad':'TIPO_ACTIVIDAD',
                                             'codigo_agenda':'CODIGO_AGENDA',
                                             'nombre_agenda_y':'NOMBRE_AGENDA',
                                             'estado_agenda':'ESTADO_AGENDA',
                                             'hora_inicio_actividad':'HORA_INICIO_ACTIVIDAD',
                                             'hora_fin_actividad':'HORA_FIN_ACTIVIDAD',
                                             'sede':'SEDE',
                                             'sesion':'SESION',
                                             'numero_sesiones':'NUMERO_SESIONES',
                                             'nombre_facilitador_y':'NOMBRE_FACILITADOR',
                                             'control_asistencia':'CONTROL_ASISTENCIA',
                                             'fecha_inicio_agenda':'FECHA_REGISTRO_ASISTENCIA',
                                             'usuario_que_programa_la_actividad':'USUARIO_REGISTRA_ASISTENCIA',
                                             'porcentaje_asistencia':'PORCENTAJE_ASISTENCIA',
                                             'macrosector_actividad':'Macrosector', 
                                             'nombre_municipio':'NOMBRE_MUNICIPIO',
                                             'programa':'PROGRAMA AL CUAL SE CARGA',
                                             'area_tematica':'ÁREA TEMÁTICA',
                                             'MODALIDAD DE PRESTACIÓN':'MODALIDAD DE PRESTACIÓN',
                                             'fechaInicio':'fecha_inicio_actividad',
                                             'fechaFin':'fecha_fin_actividad',
                                             'programa':'PROGRAMA AL CUAL SE CARGA',
                                             'fechaInicio':'fecha_inicio_actividad',
                                             'fechaFin':'fecha_fin_actividad',
                                             'LUGAR DE PRESTACIÓN':'?LUGAR'})

In [25]:
actividades[['fecha_fin_actividad','fecha_inicio_actividad']]=actividades[['fecha_fin_actividad','fecha_inicio_actividad']].apply(pd.to_datetime,format='%Y-%m-%d', errors='coerce')
actividades.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43610 entries, 0 to 43657
Data columns (total 96 columns):
 #   Column                                          Non-Null Count  Dtype         
---  ------                                          --------------  -----         
 0   NOMBRE_USUARIO                                  27921 non-null  object        
 1   NUM_DOCUMENTO                                   43610 non-null  object        
 2   TELEFONO_UNO                                    22837 non-null  object        
 3   CELULAR                                         23706 non-null  object        
 4   CORREO_ELECTRONICO                              27417 non-null  object        
 5   NOMBRE_MUNICIPIO                                27850 non-null  object        
 6   RAZON_SOCIAL                                    27921 non-null  object        
 7   NIT_EMPRESA                                     11324 non-null  float64       
 8   CODIGO_ACTIVIDAD                              

In [26]:
actividades['Año'] = actividades['Fecha de inicio actividad arreglada'].dt.year
actividades['Mes'] = actividades['Fecha de inicio actividad arreglada'].dt.month

In [27]:
## Creamos las variables faltantes
#Vacías:
actividades['APELLIDO_USUARIO_1']=""
actividades['APELLIDO_USUARIO_2']=""
actividades['APELLIDOS_FACILITADOR']=""
actividades['DOCUMENTO_FACILITADOR']=""
actividades['LINEA_DEL_CONFERENCISTA']=""
actividades['ID_DEL_DIAGNOSTICO']=""
actividades['Convenio']=""
actividades['TELEFONO_DOS']=""

In [28]:
## Calculamos las que se repiten
actividades['EJE_TEMATICO']=actividades['ÁREA TEMÁTICA']
actividades['MACRO AL CUAL SE CARGA']=actividades['Macrosector']

In [29]:
actividades['?LUGAR'].value_counts()

Online             27339
Virtual             6940
Presencial          5331
Sede Salitre        1417
Sede Chapinero      1058
Innovalab            815
Sede Fusagasugá      264
Sede Chía            179
Sede Kennedy          63
Choachí               49
Sopó                  34
Cajicá                28
Chipaque              27
Zipaquirá             18
Sede Soacha           16
Sibaté                12
Lenguazaque           10
Tibacuy                8
Villapinzón            1
Chía                   1
Name: ?LUGAR, dtype: int64

In [30]:
## Unificamos región
actividades['?LUGAR'] = actividades['?LUGAR'].str.upper().str.replace(r'[^\w\s]', '').str.strip()

actividades.loc[actividades['?LUGAR'].str.contains('ONLINE', case=False, regex=True), '?LUGAR'] = 'NA'
actividades.loc[actividades['?LUGAR'].str.contains('VIRTUAL', case=False, regex=True), '?LUGAR'] = 'NA'
actividades.loc[actividades['?LUGAR'].str.contains('OTRO ALIADO BOGOTÁ', case=False, regex=True), '?LUGAR'] = 'BOGOTÁ D.C.'
actividades.loc[actividades['?LUGAR'].str.contains('PRESENCIAL', case=False, regex=True), '?LUGAR'] = 'BOGOTÁ D.C.'
actividades.loc[actividades['?LUGAR'].str.contains('SALITRE', case=False, regex=True), '?LUGAR'] = 'BOGOTÁ D.C.'
actividades.loc[actividades['?LUGAR'].str.contains('CHAPINERO', case=False, regex=True), '?LUGAR'] = 'BOGOTÁ D.C.'
actividades.loc[actividades['?LUGAR'].str.contains('INNOVALAB', case=False, regex=True), '?LUGAR'] = 'BOGOTÁ D.C.'
actividades.loc[actividades['?LUGAR'].str.contains('KENNEDY', case=False, regex=True), '?LUGAR'] = 'BOGOTÁ D.C.'
actividades.loc[actividades['?LUGAR'].str.contains('ARBEL', case=False, regex=True), '?LUGAR'] = 'ARBELÁEZ (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('CABRER', case=False, regex=True), '?LUGAR'] = 'CABRERA (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('CAJIC', case=False, regex=True), '?LUGAR'] = 'CAJICÁ (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('QUEZA', case=False, regex=True), '?LUGAR'] = 'CAQUEZA (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('CARUPA', case=False, regex=True), '?LUGAR'] = 'CARMEN DE CARUPA (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('CHÍA', case=False, regex=True), '?LUGAR'] = 'CHÍA (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('CHIPAQUE', case=False, regex=True), '?LUGAR'] = 'CHIPAQUE (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('CHOACH', case=False, regex=True), '?LUGAR'] = 'CHOACHÍ (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('CHOCONT', case=False, regex=True), '?LUGAR'] = 'CHOCONTÁ (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('CHOCONT', case=False, regex=True), '?LUGAR'] = 'CHOCONTÁ (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('COGU', case=False, regex=True), '?LUGAR'] = 'COGUA (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('COT', case=False, regex=True), '?LUGAR'] = 'COTA (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('CUCUNU', case=False, regex=True), '?LUGAR'] = 'CUCUNUBÁ (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('MEQUE', case=False, regex=True), '?LUGAR'] = 'FÓMEQUE (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('FOSC', case=False, regex=True), '?LUGAR'] = 'FOSCA (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('QUENE', case=False, regex=True), '?LUGAR'] = 'FÚQUENE (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('FUSAGA', case=False, regex=True), '?LUGAR'] = 'FUSAGASUGÁ (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('GACHAL', case=False, regex=True), '?LUGAR'] = 'GACHALÁ (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('GACHANCI', case=False, regex=True), '?LUGAR'] = 'GACHANCIPÁ (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('GACHET', case=False, regex=True), '?LUGAR'] = 'GACHETÁ (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('GAMA', case=False, regex=True), '?LUGAR'] = 'GAMA (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('GRANA', case=False, regex=True), '?LUGAR'] = 'GRANADA (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('GUACHE', case=False, regex=True), '?LUGAR'] = 'GUACHETÁ (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('GUASCA', case=False, regex=True), '?LUGAR'] = 'GUASCA (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('GUATAVI', case=False, regex=True), '?LUGAR'] = 'GUATAVITA (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('GUAYAB', case=False, regex=True), '?LUGAR'] = 'GUAYABETAL (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('GUTI', case=False, regex=True), '?LUGAR'] = 'GUTIÉRREZ (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('JUN', case=False, regex=True), '?LUGAR'] = 'JUNÍN (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('CALER', case=False, regex=True), '?LUGAR'] = 'LA CALERA (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('LENGUAZAQUE', case=False, regex=True), '?LUGAR'] = 'LENGUAZAQUE (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('MACHE', case=False, regex=True), '?LUGAR'] = 'MACHETÁ (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('MANT', case=False, regex=True), '?LUGAR'] = 'MANTA (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('MEDIN', case=False, regex=True), '?LUGAR'] = 'MEDINA (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('NEMOC', case=False, regex=True), '?LUGAR'] = 'NEMOCÓN (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('PAND', case=False, regex=True), '?LUGAR'] = 'PANDI (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('PASC', case=False, regex=True), '?LUGAR'] = 'PASCA (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('QUETAM', case=False, regex=True), '?LUGAR'] = 'QUETAME (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('BERNARD', case=False, regex=True), '?LUGAR'] = 'SAN BERNARDO (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('SESQUI', case=False, regex=True), '?LUGAR'] = 'SESQUILÉ (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('SIBAT', case=False, regex=True), '?LUGAR'] = 'SIBATÉ (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('SILVAN', case=False, regex=True), '?LUGAR'] = 'SILVANIA (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('SIMIJA', case=False, regex=True), '?LUGAR'] = 'SIMIJACA (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('SOACHA', case=False, regex=True), '?LUGAR'] = 'SOACHA (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('SOP', case=False, regex=True), '?LUGAR'] = 'SOPÓ (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('SUESC', case=False, regex=True), '?LUGAR'] = 'SUESCA (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('SUSA', case=False, regex=True), '?LUGAR'] = 'SUSA (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('SUTATA', case=False, regex=True), '?LUGAR'] = 'SUTATAUSA (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('TABI', case=False, regex=True), '?LUGAR'] = 'TABIO (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('TAUS', case=False, regex=True), '?LUGAR'] = 'TAUSA (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('TENJ', case=False, regex=True), '?LUGAR'] = 'TENJO (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('TIBACU', case=False, regex=True), '?LUGAR'] = 'TIBACUY (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('TIBIRI', case=False, regex=True), '?LUGAR'] = 'TIBIRITÁ (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('TOCANCI', case=False, regex=True), '?LUGAR'] = 'TOCANCIPA (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('UBAL', case=False, regex=True), '?LUGAR'] = 'UBALÁ (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('UBAQ', case=False, regex=True), '?LUGAR'] = 'UBAQUE (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('UBAT', case=False, regex=True), '?LUGAR'] = 'UBATÉ (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('UNE', case=False, regex=True), '?LUGAR'] = 'UNE (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('VENEC', case=False, regex=True), '?LUGAR'] = 'VENECIA (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('VILLAP', case=False, regex=True), '?LUGAR'] = 'VILLAPINZÓN (CUNDINAMARCA)'
actividades.loc[actividades['?LUGAR'].str.contains('ZIPAQUIR', case=False, regex=True), '?LUGAR'] = 'ZIPAQUIRÁ (CUNDINAMARCA)'

C:\Users\1032452285\AppData\Local\Temp\ipykernel_5928\3666918495.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  actividades['?LUGAR'] = actividades['?LUGAR'].str.upper().str.replace(r'[^\w\s]', '').str.strip()


In [31]:
actividades['?LUGAR'].value_counts()

NA                            34279
BOGOTÁ D.C.                    8684
FUSAGASUGÁ (CUNDINAMARCA)       264
CHÍA (CUNDINAMARCA)             180
CHOACHÍ (CUNDINAMARCA)           49
SOPÓ (CUNDINAMARCA)              34
CAJICÁ (CUNDINAMARCA)            28
CHIPAQUE (CUNDINAMARCA)          27
ZIPAQUIRÁ (CUNDINAMARCA)         18
SOACHA (CUNDINAMARCA)            16
SIBATÉ (CUNDINAMARCA)            12
LENGUAZAQUE (CUNDINAMARCA)       10
TIBACUY (CUNDINAMARCA)            8
VILLAPINZÓN (CUNDINAMARCA)        1
Name: ?LUGAR, dtype: int64

In [32]:
actividades=actividades [['NOMBRE_USUARIO','APELLIDO_USUARIO_1','APELLIDO_USUARIO_2','NUM_DOCUMENTO','TELEFONO_UNO',
                        'CELULAR','CORREO_ELECTRONICO','NOMBRE_MUNICIPIO','RAZON_SOCIAL','NIT_EMPRESA','CODIGO_ACTIVIDAD',
                        'NOMBRE_ACTIVIDAD','TIPO_ACTIVIDAD','EJE_TEMATICO','CODIGO_AGENDA','NOMBRE_AGENDA','ESTADO_AGENDA',
                        'HORA_INICIO_ACTIVIDAD','HORA_FIN_ACTIVIDAD','SEDE','SESION','NUMERO_SESIONES','NOMBRE_FACILITADOR',
                        'APELLIDOS_FACILITADOR','DOCUMENTO_FACILITADOR','CONTROL_ASISTENCIA','FECHA_REGISTRO_ASISTENCIA',
                        'USUARIO_REGISTRA_ASISTENCIA','PORCENTAJE_ASISTENCIA','LINEA_DEL_CONFERENCISTA','ID_DEL_DIAGNOSTICO',
                        'Macrosector','Año','Mes','?LUGAR','PROGRAMA AL CUAL SE CARGA','MACRO AL CUAL SE CARGA','ÁREA TEMÁTICA',
                        'MODALIDAD DE PRESTACIÓN','Convenio','TELEFONO_DOS','fecha_inicio_actividad','fecha_fin_actividad']] 

**Arreglos** 

In [33]:
actividades = actividades.rename(columns={'NUM_DOCUMENTO': 'documentousuario'})
actividades['documentousuario'] = actividades['documentousuario'].astype(str)
actividades = actividades[actividades['documentousuario'] != "."]

In [34]:
actividades = actividades.rename(columns={'CODIGO_AGENDA': 'codigo_agenda'})

In [35]:
actividades['_dup'] = actividades.duplicated(subset=['codigo_agenda', 'SESION', 'fecha_inicio_actividad', 'documentousuario'])
actividades = actividades.sort_values(by=['codigo_agenda', 'SESION', 'documentousuario', '_dup'])
actividades = actividades[actividades['_dup'] == False]
actividades = actividades.drop('_dup', axis=1)

In [36]:
actividades.loc[actividades['NIT_EMPRESA'].isnull(), 'NIT_EMPRESA'] = 0
actividades['nit'] = pd.to_numeric(actividades['NIT_EMPRESA'], errors='coerce')
actividades['NIT'] = actividades.groupby('documentousuario')['nit'].first().astype(int).astype(str)
actividades = actividades.drop(['NIT_EMPRESA', 'nit'], axis=1)

In [37]:
actividades.drop_duplicates(subset=['codigo_agenda', 'SESION', 'fecha_inicio_actividad', 'documentousuario'], inplace=True)

In [38]:
actividades['documentousuario'] = actividades['documentousuario'].str.replace(".", "")
actividades['year'] = 2023
actividades.sort_values(by='documentousuario', inplace=True)

C:\Users\1032452285\AppData\Local\Temp\ipykernel_5928\1046315549.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  actividades['documentousuario'] = actividades['documentousuario'].str.replace(".", "")


In [39]:
actividades.loc[actividades['NOMBRE_AGENDA'].isnull(), 'NOMBRE_AGENDA'] = actividades['NOMBRE_ACTIVIDAD']

In [40]:
actividades = actividades[~actividades['NOMBRE_AGENDA'].str.contains("rueba")]
actividades.loc[:,'PROGRAMA AL CUAL SE CARGA'] = actividades['PROGRAMA AL CUAL SE CARGA'].astype(str).replace(".", "")

In [41]:
#Para confirmar que todas las agendas tengan nombre de facilitador
actividades['NOMBRE_FACILITADOR'].value_counts(dropna=False)

OLGA LUCIA VALLEJO KATTAH            20441
MUÑOZ GUERRERO JORGE ANDRES           6940
PEÑA GUEVARA ALBA TATIANA             3784
RIBERO LUNA LUZ YANETH                1432
USA BUSINESS                          1013
                                     ...  
DESARROLLADOR COMERCIAL TCF GROUP        9
BANCO POPULAR                            8
BANCO BBVA                               7
BANCO DE BOGOTÁ                          4
BANCO SERFINANZA                         4
Name: NOMBRE_FACILITADOR, Length: 78, dtype: int64

In [42]:
#Bazzarbog (meses anteriores)
bazzarbog = ['A0000028144', 'A0000028145', 'A0000013134', 'A0000015376', 'A0000015712', 'A0000016498', 'A0000016560', 'A0000017522','A0000017523',
                'A0000018696', 'A0000018697', 'A0000019314', 'A0000019315', 'A0000021009', 'A0000021010', 'A0000021060','A0000021061', 'A0000023162', 
                'A0000023163', 'A0000023217', 'A0000023218', 'A0000026992', 'A0000026993', 'A0000028643','A0000028644','A0000029394','A0000029397',
                'A0000029693','A0000029694']
actividades = actividades[~actividades['codigo_agenda'].isin(bazzarbog)]

In [43]:
#Negocia (meses anteriores)
negocia = ['A0000028140', 'A0000028141', 'A0000028142', 'A0000028143', 'A0000013135', 'A0000015357', 'A0000015358', 'A0000015359', 'A0000015702', 
           'A0000016496', 'A0000016557', 'A0000016558', 'A0000016559', 'A0000017524', 'A0000017525', 'A0000017526', 'A0000017527', 'A0000018699', 
           'A0000018700', 'A0000018702', 'A0000018698', 'A0000019316', 'A0000019317', 'A0000019318', 'A0000019319', 'A0000021003', 'A0000021004', 
           'A0000021005', 'A0000021008', 'A0000021056', 'A0000021057', 'A0000021058', 'A0000021059', 'A0000023158', 'A0000023159', 'A0000023160', 
           'A0000023161', 'A0000023213', 'A0000023214', 'A0000023215', 'A0000023216', 'A0000026987', 'A0000026988', 'A0000026989', 'A0000026991',
           'A0000029400', 'A0000029402', 'A0000029404', 'A0000029405', 'A0000029695', 'A0000029696', 'A0000029697', 'A0000029698']
actividades = actividades[~actividades['codigo_agenda'].isin(negocia)]

In [44]:
actividades['TIPO_ACTIVIDAD'] = actividades['TIPO_ACTIVIDAD'].str.upper().str.replace(r'[^\w\s]', '').str.strip()

actividades.loc[actividades['TIPO_ACTIVIDAD'].str.contains('GRUPAL', case=False, regex=True), 'TIPO_ACTIVIDAD'] = 'ASESORIA GRUPAL'
actividades.loc[((actividades['NOMBRE_ACTIVIDAD'].str.contains('Agenda de contacto con inversionistas', case=False, regex=True)) | 
        (actividades['NOMBRE_ACTIVIDAD'].str.contains('Mentoría con Inversión', case=False, regex=True))) &
       (actividades['TIPO_ACTIVIDAD'].str.contains('ENCUENTRO FINANCIERO', case=False, regex=True)), 'TIPO_ACTIVIDAD'] = 'AGENDA CON INVERSIONISTAS'
actividades.loc[actividades['TIPO_ACTIVIDAD'].str.contains('WEBINAR', case=False, regex=True), 'TIPO_ACTIVIDAD'] = 'EVENTO MASIVO'
actividades.loc[actividades['NOMBRE_ACTIVIDAD'].str.contains('Mixer de Innovación', case=False, regex=True), 'TIPO_ACTIVIDAD'] = 'NETWORKING'
actividades.loc[actividades['TIPO_ACTIVIDAD'].str.contains('CÁPSULAS', case=False, regex=True), 'TIPO_ACTIVIDAD'] = 'CÁPSULA DE CONOCIMIENTO'
actividades.loc[actividades['TIPO_ACTIVIDAD'].str.contains('PLATAFORMA DIGITAL', case=False, regex=True) &
       ~actividades['codigo_agenda'].isin(['A0000026992', 'A0000026993', 'A0000026987', 'A0000026988', 'A0000026989', 'A0000026991']),
       'TIPO_ACTIVIDAD'] = 'EVENTO DE CONTACTO COMERCIAL'
actividades.loc[actividades['codigo_agenda'].isin(['A0000026992', 'A0000026993', 'A0000026987', 'A0000026988', 'A0000026989', 'A0000026991']),
       'TIPO_ACTIVIDAD'] = 'PLATAFORMA DIGITAL'
actividades.loc[actividades['NOMBRE_ACTIVIDAD'].str.contains('Agenda contacto financiero', case=False, regex=True), 'TIPO_ACTIVIDAD'] = 'AGENDA CON BANCOS'
actividades.loc[actividades['NOMBRE_ACTIVIDAD'].str.contains('Demo day', case=False, regex=True), 'TIPO_ACTIVIDAD'] = 'INVESTOR DAY'
actividades.loc[actividades['TIPO_ACTIVIDAD'] == 'SESIÓN TALLER', 'TIPO_ACTIVIDAD'] = 'TALLER'

C:\Users\1032452285\AppData\Local\Temp\ipykernel_5928\1344023353.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  actividades['TIPO_ACTIVIDAD'] = actividades['TIPO_ACTIVIDAD'].str.upper().str.replace(r'[^\w\s]', '').str.strip()


In [45]:
## Para confirmar que todos los tipos de actividad estén correctos
actividades['TIPO_ACTIVIDAD'].value_counts(dropna=False)

TALLER                          9108
EVENTO MASIVO                   7873
EVENTO DE CONTACTO COMERCIAL    6745
ASESORIA GRUPAL                 2344
CÁPSULA DE CONOCIMIENTO         1493
PROGRAMA ESPECIALIZADO          1033
CONSULTORÍA ESPECIALIZADA        682
AGENDA CON BANCOS                481
RUEDA DE NEGOCIOS                190
INVESTOR DAY                      99
NETWORKING                        98
ESCENARIOS DE ARTICULACIÓN        44
AGENDA CON INVERSIONISTAS          3
Name: TIPO_ACTIVIDAD, dtype: int64

In [46]:
#Para confirmar que todas los nombres del programa esten correctos
actividades['PROGRAMA AL CUAL SE CARGA'].value_counts(dropna=False)

SOLUCIONES EMPRESARIALES                    15870
INNOVACIÓN                                   4407
CRECIMIENTO PARA LA FORMALIZACIÓN            3519
FORO DE PRESIDENTES                          2525
EMPRENDIMIENTO                               2088
CONSOLIDACIÓN Y ESCALAMIENTO EMPRESARIAL     1784
Name: PROGRAMA AL CUAL SE CARGA, dtype: int64

In [47]:
actividades['PORCENTAJE_ASISTENCIA'].value_counts(dropna=False)

100.00    29472
50.00       707
14.29        14
Name: PORCENTAJE_ASISTENCIA, dtype: int64

In [48]:
actividades = actividades.sort_values(by='Mes')
actividades = actividades[actividades['PORCENTAJE_ASISTENCIA'] != 0]

**Guardar las bases**

In [49]:
actividades.to_excel('C:/Users/1032452285/OneDrive - Camara de Comercio de Bogota/Documentos - Gestión Analítica VFE/Mensuales/5. Cierre de cifras/2. Archivos limpios/Asistencias 2023.xlsx', index = False)
actividades.to_excel('C:/Users/1032452285/OneDrive - Camara de Comercio de Bogota/Documentos - Gestión Analítica VFE/Bases/Asistencias/Asistencias 2023.xlsx', index = False)
actividades.to_csv('C:/Users/1032452285/OneDrive - Camara de Comercio de Bogota/Documentos - Gestión Analítica VFE/Mensuales/5. Cierre de cifras/4. Bases de datos/Asistencias_2023.csv')

# Beneficiarios Internacionalización

In [50]:
internacionalizacion=pd.read_excel('C:/Users/1032452285/OneDrive - Camara de Comercio de Bogota/Documentos - Gestión Analítica VFE/Mensuales/2. Planeación/Soporte Internacionalización/Agendas Internacionalización - I Trimestre VFE.xlsx')

In [52]:
internacionalizacion=internacionalizacion.drop_duplicates(subset=['Código de agenda'], keep='last')

In [53]:
beneficiarios=internacionalizacion.merge(actividades, left_on='Código de agenda', right_on='codigo_agenda', how='outer', indicator = True)

In [54]:
beneficiarios['_merge'].value_counts()

right_only    28987
both           1206
left_only         1
Name: _merge, dtype: int64

In [55]:
_merge=['both']
beneficiarios=beneficiarios.loc[beneficiarios['_merge'].isin(_merge)]
beneficiarios['_merge'].value_counts()

both          1206
left_only        0
right_only       0
Name: _merge, dtype: int64

In [56]:
beneficiarios=beneficiarios.drop_duplicates(subset=['documentousuario'], keep='last')

In [57]:
beneficiarios.to_excel('C:/Users/1032452285/OneDrive - Camara de Comercio de Bogota/Documentos - Gestión Analítica VFE/Mensuales/2. Planeación/Soporte Internacionalización/Beneficiarios 2023-I.xlsx', index = False)